In [1]:
%load_ext lineapy

In [27]:
import git
import pprint

In [29]:
repo = git.Repo()
repo_info = dict(
    repo_name = repo.working_tree_dir.split("/")[-1],
    repo_branch = repo.active_branch.name,
    repo_commit = str(repo.active_branch.commit),
    repo_diff = [{'a_path':x.a_path, 'b_path':x.b_path, 'change_type':x.change_type} for x in repo.index.diff(None)]
)
lineapy.save(repo_info,'repo_info')
pprint.pprint(repo_info)

{'repo_branch': 'main',
 'repo_commit': '9451e1fe91ab3f6b33968204cf19efd888ff12e6',
 'repo_diff': [{'a_path': 'R/R_workflow.ipynb',
                'b_path': 'R/R_workflow.ipynb',
                'change_type': 'M'},
               {'a_path': 'cpp/cpp_workflow.ipynb',
                'b_path': 'cpp/cpp_workflow.ipynb',
                'change_type': 'M'},
               {'a_path': 'quantlib/quantlib_1.27.ipynb',
                'b_path': 'quantlib/quantlib_1.27.ipynb',
                'change_type': 'M'},
               {'a_path': 'quantlib/quantlib_get_function_1.25.ipynb',
                'b_path': 'quantlib/quantlib_get_function_1.25.ipynb',
                'change_type': 'M'},
               {'a_path': 'report_sample.ipynb',
                'b_path': 'report_sample.ipynb',
                'change_type': 'M'}],
 'repo_name': 'LineaPyModelValidation'}


In [21]:
import datetime
print('current UTC:', datetime.datetime.utcnow())

current UTC: 2022-10-12 21:22:40.965753


In [2]:
# quantlib v1.25

In [14]:
quantlib_version = lineapy.get('quantlib_version')
print(quantlib_version.get_value())

1.25


In [3]:
quantlib_price = lineapy.get('quantlib_price')
print(quantlib_price.get_value())

10.418271653959474


In [4]:
print(quantlib_price.get_code())

import datetime
import json
import sys

cpp_config_path = "../config/option_config.json"
params = json.load(open(cpp_config_path, "r"))


def load_quantlib(version="1.25"):
    sys.path.append(f"/home/mingjerli/quantlib/QuantLib-{version}")
    import QuantLib

    return QuantLib


version = "1.25"
ql = load_quantlib(version)
today = datetime.datetime.today()
maturity_date = today + datetime.timedelta(365 * params["T"])
maturity_date = ql.Date(maturity_date.day, maturity_date.month, maturity_date.year + 1)
strike_price = params["K"]
option_type = ql.Option.Call
payoff = ql.PlainVanillaPayoff(option_type, strike_price)
exercise = ql.EuropeanExercise(maturity_date)
european_option = ql.VanillaOption(payoff, exercise)
bs_price = european_option.NPV()



In [5]:
# Benchmark Model C++

In [6]:
cpp_price = lineapy.get('cpp_price')
print(cpp_price.get_value())

10.98079096253393


In [7]:
cpp_source_code = lineapy.get('cpp_config')
# print(cpp_source_code.get_value())
print(cpp_source_code.get_code())

import json

cpp_config_path = "../config/option_config.json"
params = json.load(open(cpp_config_path, "r"))



In [8]:
cpp_source_code = lineapy.get('cpp_source_code')
# print(cpp_source_code.get_value())
print(cpp_source_code.get_code())

import pathlib

source_code_path = "lib.cpp"
source_code = pathlib.Path(source_code_path).read_text()



In [9]:
# Benchmark Model R 

In [10]:
R_price = lineapy.get('R_price')
print(R_price.get_value())

10.4035391529966


In [11]:
R_source_code = lineapy.get('R_source_code')
print(R_source_code.get_value())
# print(R_source_code.get_code())

library(OptionPricing)
library(jsonlite)

config = fromJSON("../config/option_config.json")
european_call = BS_EC(K=config$K, r=config$r, sigma=config$v, T=config$T, S0=config$S)
print(european_call)
write.csv(european_call, file='R_call_price.csv')


In [12]:
# Benchmark Model quantlib v2

In [15]:
benchmark_quantlib_version = lineapy.get('benchmark_quantlib_version')
print(benchmark_quantlib_version.get_value())

1.25


In [16]:
benchmark_quantlib_price = lineapy.get('benchmark_quantlib_price')
print(benchmark_quantlib_price.get_value())

10.418271653959474


In [17]:
print(benchmark_quantlib_price.get_code())

ft = lineapy.get_function(
    ["quantlib_price", "quantlib_version"], input_parameters=["version"]
)
benchmark_quantlib_result = ft("1.25")
benchmark_quantlib_price = benchmark_quantlib_result["quantlib_price"]



In [13]:
# Benchmark Model Excel